In [1]:
import xlwt
import pandas as pd
from datetime import date

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
raw_file = '../data/doctoralia_raw.csv'
output_file = '../data/doctoralia_tidy.csv'
xls_file = '../data/' + date.today().strftime('%Y%m%d') + '_Dados_Doctoralia.xls'
sheet_name = 'Planilha'

In [4]:
id_col = 'nome'
list_cols = ['endereco', 'telefone', 'especialidade']
max_exploded_cols = 3
nan_threshold = 0.90

***

In [5]:
data = pd.read_csv(raw_file, sep=';')

In [6]:
data.head()

,nome,endereco,telefone,especialidade,nota,registro
0,Marisabel Boere de Moraes Reis,R. Dr. Osvaldo Ribeiro 198 sala303 Empresarial Mar à Vista Ondina Salvador,(71) 4102-1149,Ginecologista,5.0,Número de registro:Crm 1654-6
1,Sofia Andrade,"Avenida Miguel Calmon 40 (Edf. Conde dos Arcos sala 102-103) Comercio Salvador,Av. Adhemar de Barros 67 Ondina Salvador","(71) 3245-4009,(71) 3012-3010,(71) 3247-4009",Ginecologista,5.0,Número de registro:23512 BA
2,Anabel Silva Platon Bezerra,Rua da Graça 374 Graça Salvador,(71) 3336-2850,Ginecologista,5.0,Número de registro:CRM 8224 BARQE Nº: 3626
3,Cristina Fernandes Noronha,Av. Juracy Magalhães Junior 2096 (Sala 209) Rio Vermelho Salvador,"(71) 3355-0786,(71) 2108-4684",Ginecologista,5.0,Número de registro:CRM 11896 BA - RQE 7062
4,Pedro Paulo Bastos Filho,"Av. Jequitaia 40 (Edf. Conde dos Arcos sala 102-103) Mares Salvador,Avenida Anita Garibaldi 1555 sala-805 federação Engenho Velho Da Federacao Salvador","(71) 98233-2613,(71) 3012-3010,(71) 3247-5522",Ginecologista,5.0,Número de registro:CRM 7781-BA


In [7]:
data.shape

(26231, 6)

Dropping duplicates:

In [8]:
data = data.drop_duplicates()

Exploding list columns:

In [9]:
for col in list_cols:
    exploded_raw = data[col].str.split(',', expand=True)
    exploded_cols = [col + '_' + str(ind+1) for ind, _ in zip(range(max_exploded_cols), exploded_raw.columns)]
    
    data[exploded_cols] = exploded_raw.iloc[:, :len(exploded_cols)]
    data.drop(columns=col, inplace=True)
    if data[exploded_cols[-1]].isna().sum() > nan_threshold:
        data.drop(columns=exploded_cols[-1], inplace=True)

In [10]:
data.head(3)

,nome,nota,registro,endereco_1,endereco_2,telefone_1,telefone_2,especialidade_1,especialidade_2
0,Marisabel Boere de Moraes Reis,5.0,Número de registro:Crm 1654-6,R. Dr. Osvaldo Ribeiro 198 sala303 Empresarial Mar à Vista Ondina Salvador,None,(71) 4102-1149,None,Ginecologista,None
1,Sofia Andrade,5.0,Número de registro:23512 BA,Avenida Miguel Calmon 40 (Edf. Conde dos Arcos sala 102-103) Comercio Salvador,Av. Adhemar de Barros 67 Ondina Salvador,(71) 3245-4009,(71) 3012-3010,Ginecologista,None
2,Anabel Silva Platon Bezerra,5.0,Número de registro:CRM 8224 BARQE Nº: 3626,Rua da Graça 374 Graça Salvador,None,(71) 3336-2850,None,Ginecologista,None


Removing unwanted observations:

In [11]:
# Doctors without main telephone number
data = data[data.telefone_1.notna()]

In [12]:
# Doctors whose main phone numbers are from other states (MG, PE, RJ etc)
data = data[~data.telefone_1.str.contains(pat=r'^\([^7]\d\).*', na=False)].reset_index(drop=True)

In [13]:
# Doctors outside Bahia
data = data[~data.registro.str.contains(r'.*MG.*', na=False) | \
            data.registro.str.contains(r'.*BA.*', na=False) | \
            ~data.telefone_1.str.contains(pat=r'^\([^7]\d\).*', na=True)].reset_index(drop=True)

Tidying columns:

In [14]:
ordered_cols = [col for col in data.columns if col.startswith('especialidade')] + \
               ['nome', 'registro'] + \
               [col for col in data.columns if col.startswith('telefone')] + \
               [col for col in data.columns if col.startswith('endereco')] + \
               ['nota']

formatted_cols = [col.title().replace('_', ' ') for col in ordered_cols]

Reordering columns and sorting rows:

In [15]:
especialidades = [col for col in data.columns if col.startswith('especialidade')]

data = data[ordered_cols].sort_values(by=especialidades).reset_index(drop=True)

In [16]:
data.head(3)

,especialidade_1,especialidade_2,nome,registro,telefone_1,telefone_2,endereco_1,endereco_2,nota
0,Alergista,Anestesiologista,Moises Candido Neri Barreto,Número de registro:CRM 5874 SECRM 10846 BA - RQE 3857,(75) 3421-3529,None,Alergia e Imunologia Anestesiologia Alagoinhas,None,NaN
1,Alergista,Cardiologista,Zuleide Oliveira Nicacio,Número de registro:CRM 18619 PE,(75) 3281-3777,None,Manoel Novaes 00 - S/N Paulo Afonso,None,NaN
2,Alergista,Dermatologista,Luciano José F Guimarães,NaN,(71) 3322-2836,None,Av Sete de Setembro 57 (Ed. Fernandez 1º Andar S/104105) Centro Salvador,None,5.0


In [17]:
data.shape

(16322, 9)

Saving clean data to a .csv file:

In [18]:
data.to_csv(output_file, sep=';', index=False)

Formatting columns' names and exporting to a.xls file:

In [22]:
title = 'Profissionais de Saúde - BA'

link = 'https://www.doctoralia.com.br/'
subtitle = xlwt.Formula('concatenate("Fonte: ", HYPERLINK("{}"))'.format(link))

rows = {'title': 0, 'subtitle': 1, 'blank': 2, 'header': 3}

In [23]:
def format_sheet(worksheet, num_cols):

    # NOTE: height = font_size * 20
    style_title = 'font: name Arial, bold on, height 320; align: horiz left'
    style_subtitle = 'font: name Arial, bold on, height 200; align: horiz left'
    
    # NOTE: width = num_chars * 256
    max_width = max([len(col) for col in formatted_cols]) * 256
    for i in range(num_cols):
        worksheet.col(i).width = max_width

    worksheet.write(r=rows['title'], c=0, label=title, style=xlwt.easyxf(style_title))
    worksheet.row(rows['title']).height = 360
    
    worksheet.write(r=rows['subtitle'], c=0, label=subtitle, style=xlwt.easyxf(style_subtitle))
    worksheet.row(rows['subtitle']).height = 340

    worksheet.merge(r1=rows['title'], c1=0, r2=rows['title'], c2=num_cols-1)    
    worksheet.merge(r1=rows['subtitle'], c1=0, r2=rows['subtitle'], c2=num_cols-1)
    worksheet.merge(r1=rows['blank'], c1=0, r2=rows['blank'], c2=num_cols-1)


In [24]:
data.rename(columns=dict(zip(ordered_cols, formatted_cols)), inplace=True)

with pd.ExcelWriter(xls_file, engine='xlwt', mode='w') as writer:
    data.to_excel(writer, sheet_name=sheet_name, startrow=rows['header'], encoding='utf-8', index=False)
    
    worksheet = writer.sheets[sheet_name]
    format_sheet(worksheet, len(data.columns))